In [1]:
import os
os.chdir("/root/data/DBP_sa_bc/")
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import anndata as ad
import pandas as pd
import copy

In [2]:
parser = argparse.ArgumentParser()
# parser.add_argument('--tasks', type=str, nargs='+',  default=["dogma_full", "dogma_paired_full", 
#     "dogma_paired_abc", "dogma_paired_ab",  "dogma_paired_ac", "dogma_paired_bc",
#     "dogma_single_full", "dogma_single"])
parser.add_argument('--tasks', type=str, nargs='+',  default=["lung_ts"])
parser.add_argument('--method', type=str, default='DBP_sa_bc')
# parser.add_argument('--mosaic', type=int, default=1)
parser.add_argument('--experiment', type=str, default='e52')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--init_model', type=str, default='sp_latest')
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [3]:
# midas_embed results
df_batch_bio_embed = {}
o.tasks = [o.tasks[0]]
for task in o.tasks:
    df_batch_bio_embed[task] = pd.read_excel(pj("result", "comparison", task, o.method, o.experiment, o.init_model, "metrics_fa.xlsx"))
    df_batch_bio_embed[task].rename(index={0: task}, inplace=True)
df_batch_bio_embed_cat = pd.concat(df_batch_bio_embed.values(), axis=0)

# df_batch_bio_embed_cat[["task"]] = df_batch_bio_embed_cat.index
df_batch_bio_embed_cat.rename(index={i: "DBP" for i in df_batch_bio_embed_cat.index}, inplace=True)
df_batch_bio_embed_cat


,l2_l1,k4,l0,distances
DBP,0.012944,66.298203,0.579838,0.194615


In [4]:
# sota results
methods = [
    # "harmony",
    "liger",
    "mofa",
    # "scanorama_embed",
    # "seurat_cca",
    # "seurat_rpca",
    "LDVAE",
    "scETM"
    # "pca",
]

df_sota = {}
for method in methods:
    if "DBP_sa_bc" in method:
        df_sota[method] = pd.read_excel(pj("result", "comparison", o.tasks[0], method, o.experiment, o.init_model, "metrics_fa.xlsx"))
    else:
        df_sota[method] = pd.read_excel(pj("result", "comparison", o.tasks[0], method, "metrics_fa.xlsx"))
    df_sota[method].rename(index={0: method}, inplace=True)
df_sota_cat = pd.concat(df_sota.values(), axis=0)

# df_sota_cat[["task"]] = o.tasks
# df_sota_cat


In [5]:
df_cat = pd.concat([df_batch_bio_embed_cat, df_sota_cat], axis=0)
df_cat = (df_cat-df_cat.min(axis=0))/(df_cat.max(axis=0)-df_cat.min(axis=0))

df_mean_cat = copy.deepcopy(df_cat)
df_mean_cat["mean_score"] = df_cat[["l2_l1", "k4", "l0"]].mean(axis=1)
# df_mean_cat["bio_score"] = df_cat[["label_ASW", "il_score_ASW", "il_score_f1", "NMI", "ARI", "cLISI"]].mean(axis=1)
# df_mean_cat["overall_score"] = 0.4 * df_mean_cat["batch_score"] + 0.6 * df_mean_cat["bio_score"]
# df_mean_cat = df_mean_cat[["graph_conn", "batch_ASW", "kBET", "iLISI", "batch_score", "label_ASW", "il_score_ASW", "il_score_f1", "NMI", "ARI", "cLISI", "bio_score", "overall_score"]]
df_mean_cat.sort_values("mean_score", ascending=False, inplace=True)
# df_mean_cat
df_mean_cat

,l2_l1,k4,l0,distances,mean_score
DBP,0.894909,1.000000,1.00000,1.000000,0.973727
liger,1.000000,0.666401,0.61546,0.316195,0.649514
mofa,0.292310,0.421385,0.00000,0.000000,0.178424
LDVAE,0.074833,0.000000,0.00000,0.138186,0.053255
scETM,0.000000,0.029830,0.00000,0.172612,0.050610


In [6]:
out_dir = pj("eval", "plot", "data")
utils.mkdir(out_dir, remove_old=False)
# df_mean_cat.to_excel(pj(out_dir, "scib_metrics_dbp_"+o.tasks[0].split("_")[0]+"_"+o.init_model+".xlsx"))
df_mean_cat.to_excel(pj(out_dir, "fa_metrics_"+o.tasks[0]+"_"+o.method+"_"+o.experiment+"_"+o.init_model+".xlsx"))